In [16]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import pycrfsuite

from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report

def readCorpus(fpath):
    sents = []
    with open(fpath) as fd:
        sent = []
        for l in fd:
            #lt = l.strip().decode("utf8")
            lt = l.strip()
            if not lt:
                sents.append(sent)
                sent = []
            else:
                w_t = lt.split('\t')
                sent.append([w_t[0], w_t[1]])
    return sents

def word2features(sent,i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        #'bias',
        'word.lower=' + word.lower(),
        #'word[-3:]=' + word[-3:],
        #'word[-2:]=' + word[-2:],
        #'word.isupper=%s' % word.isupper(),
        #'word.istitle=%s' % word.istitle(),
        #'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        #'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        pass
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            #'-1:word.istitle=%s' % word1.istitle(),
            #'-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            #'-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        pass
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            #'+1:word.istitle=%s' % word1.istitle(),
            #'+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            #'+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, label in sent]
def sent2tokens(sent):
    return [token for token, label in sent]

In [18]:
train_sents = readCorpus("CrfTrain.TXT")
#print(train_sents)
x_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]
#print(y_train)

print("start append train set.")
trainer = pycrfsuite.Trainer(algorithm='l2sgd',verbose=False)
for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)
print("append train set done.")
trainer.set_params({
    #'c1': 0.01,
    'c2': 0.01,
    'max_iterations': 100,
    'feature.possible_transitions': True
    })

trainer.train("trained_model")
test_object = pycrfsuite.Tagger()
test_object.open("trained_model")


start append train set.
append train set done.


In [19]:
#Test on the first sentence of the training data
#First load the trained model
tagger = pycrfsuite.Tagger()
tagger.open('trained_model')

#Read the first sentence
#print(train_sents[0])
example_sent = train_sents[0]
print(' '.join(sent2tokens(example_sent)), end='\n\n')
print(sent2labels(example_sent))

#Predict and test
print("Predicted:", ' '.join(tagger.tag(sent2features(example_sent))))
print("Correct:  ", ' '.join(sent2labels(example_sent)))

#This will give you the precision, recall, F-score
def sentiment_classification_report(y_true, y_pred):
 
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
        
    tagset = set(lb.classes_) - {'O'}
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)}
    
    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels = [class_indices[cls] for cls in tagset],
        target_names = tagset,
    )

##Evaluation of the labeller
y_pred = [tagger.tag(xseq) for xseq in x_train]
print(sentiment_classification_report(y_train, y_pred))

same as picture but string chahi ali moto cha but nice quality

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-ENG', 'O']
Predicted: O O O O O O O O O O B-ENG O
Correct:   O O O O O O O O O O B-ENG O
              precision    recall  f1-score   support

       B-ENG       1.00      1.00      1.00       264
       I-ENG       1.00      1.00      1.00        84
       B-NEG       1.00      1.00      1.00        54
       I-NEG       1.00      1.00      1.00         6
       B-POS       1.00      1.00      1.00       210
       I-POS       1.00      1.00      1.00        45
           o       1.00      1.00      1.00         2

   micro avg       1.00      1.00      1.00       665
   macro avg       1.00      1.00      1.00       665
weighted avg       1.00      1.00      1.00       665
 samples avg       0.20      0.20      0.20       665



e:\Project Sentiment analysis\projenv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
e:\Project Sentiment analysis\projenv\Lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [20]:
# """Read txt file containing test comment"""

def read_text_file(file_path):
    words=[]  # List to store the words
    
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            line_words = line.split()  # Split line into individual words
            for word in line_words:
                words.append(word)  # Add words to the list

    
    return words


file_path = "E:/Project Sentiment analysis/CRF/xyz.txt"  # Path to the text file

# Read the text file and store words in a list
word_list = read_text_file(file_path)
print(word_list)



['wow', 'kasto', 'ramro', 'product', 'malai', 'manparyo', ':Positive', 'Sentence']


In [21]:
#Test on the test data 
#First load the trained model
tagger = pycrfsuite.Tagger()
tagger.open('trained_model')

#Predict and test
print(word_list)

new_list = [[word, ''] for word in word_list]

# example=[]
# example.append(("hi"))
# print(example)

feature=sent2features(new_list)
#print(feature)
lab=tagger.tag(feature)
print(lab)
print("Predicted:", ' '.join(lab))

PosCount=0
NegCount=0
for i in lab:
    if(i=="B-POS" or i=="I-POS"):
        PosCount+=1
    elif(i=="B-NEG" or i=="I-NEG"):
        NegCount+=1
print(f"Positive Sentiment Count={PosCount}")
print(f"Negative Sentiment Count={NegCount}")

['wow', 'kasto', 'ramro', 'product', 'malai', 'manparyo', ':Positive', 'Sentence']
['B-ENG', 'I-ENG', 'B-POS', 'I-POS', 'B-POS', 'I-POS', 'B-POS', 'I-POS']
Predicted: B-ENG I-ENG B-POS I-POS B-POS I-POS B-POS I-POS
Positive Sentiment Count=6
Negative Sentiment Count=0


In [22]:
pip install python-crfsuite scikit-learn scipy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
pip install sklearn_crfsuite

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [24]:
pip show scikit-learn

Name: scikit-learn
Version: 1.3.0
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: e:\Project Sentiment analysis\projenv\Lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [35]:
import pycrfsuite
from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report

from sklearn_crfsuite import CRF
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


# Define the parameter grid for hyperparameter optimization
param_grid = {
    #'c1': uniform(loc=0.01, scale=0.09),  # Uniform distribution between 0.01 and 0.1
    'c2': uniform(loc=0.01, scale=0.09),
    'max_iterations': [100, 200, 300]  # Test different values
    #'feature.possible_transitions': [True, False]  # Test different values
}

# Create the CRF trainer object
# trainer = CRF(algorithm='l2sgd', verbose=False)
trainer = CRF(algorithm='l2sgd')

# # Append training data
# for xseq, yseq in zip(x_train, y_train):
#     trainer.append(xseq, yseq)

# Create the RandomizedSearchCV object
random_search = RandomizedSearchCV(estimator=trainer, param_distributions=param_grid, n_iter=10, cv=3)

train_sents = readCorpus("CrfTrain.TXT")
x_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]

# Perform hyperparameter optimization
try:
    random_search.fit(x_train, y_train)
except AttributeError:
    pass

# Get the best estimator and its parameters
best_trainer = random_search.best_estimator_
best_params = random_search.best_params_
print(best_trainer)
print(best_params)


AttributeError: 'RandomizedSearchCV' object has no attribute 'best_estimator_'

In [26]:
# example3 = [['kasto',''], ['naramro',''], ['product',''], ['yo',''], 
#             ['i',''], ['dont',''], ['recommend',''], ['it','']]
# y_pred2 = crf.predict_single(sent2features(example3))
# print(y_pred2)
# y_train = sent2labels(example3)
# print(y_train)

from itertools import chain

import nltk
import sklearn
import scipy.stats
from sklearn.metrics import make_scorer
from sklearn.cross_validation import cross_val_score
from sklearn.grid_search import RandomizedSearchCV

import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

# define fixed parameters and parameters to search
crf = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=-1,
                        n_iter=50,
                        scoring=f1_scorer)
rs.fit(x_train, y_train)
 

ModuleNotFoundError: No module named 'sklearn.cross_validation'